In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import pandas as pd
import numpy as np
import os
import pickle

# =========================================================
# 1. 환경 설정 및 데이터 로드
# =========================================================
base_path = "/content/drive/MyDrive/unstructured"

# [입력 경로 확인 필요] Raw 데이터 혹은 필터링 전 데이터를 지정해주세요.
input_rating_path = f"{base_path}/rating_small_data_final.csv"
save_path = f"{base_path}/k10_filtered" # 경로 구분 위해 폴더명 살짝 변경 추천

if not os.path.exists(save_path):
    os.makedirs(save_path)

print("데이터 로드 중...")
df = pd.read_csv(input_rating_path)

print(f"원본 데이터 인터랙션 수: {len(df)}")
print(f"원본 유저 수: {df['userId'].nunique()}, 원본 아이템 수: {df['movieId'].nunique()}")

# =========================================================
# [NEW] 1.5. K-core Filtering (k=10)
# =========================================================
# 유저와 아이템 모두 최소 10개 이상의 인터랙션이 있는 데이터만 남김
# 한쪽을 지우면 다른 쪽 count가 변하므로, 변화가 없을 때까지 반복 수행해야 함

k_core = 10
print(f"\n=== {k_core}-Core Filtering 시작 ===")

while True:
    start_len = len(df)

    # 1. 유저 필터링 (count < k 제거)
    user_counts = df['userId'].value_counts()
    valid_users = user_counts[user_counts >= k_core].index
    df = df[df['userId'].isin(valid_users)]

    # 2. 아이템 필터링 (count < k 제거)
    item_counts = df['movieId'].value_counts()
    valid_items = item_counts[item_counts >= k_core].index
    df = df[df['movieId'].isin(valid_items)]

    end_len = len(df)

    print(f"필터링 진행 중... 남은 데이터: {end_len}")

    # 더 이상 제거된 데이터가 없으면 반복 종료
    if start_len == end_len:
        break

print(f"\n=== {k_core}-Core Filtering 완료 ===")
print(f"최종 인터랙션 수: {len(df)}")
print(f"최종 유저 수: {df['userId'].nunique()}, 최종 아이템 수: {df['movieId'].nunique()}")
print(f"평점 분포:\n{df['rating'].value_counts().sort_index()}")

# =========================================================
# 2. 정렬 (Sorting)
# =========================================================
if 'timestamp' in df.columns:
    df = df.sort_values(['userId', 'timestamp'])
else:
    df = df.sort_values(['userId'])

# =========================================================
# 3. 데이터 분할 (Smart Leave-one-out)
# =========================================================
# 전략: Val/Test는 무조건 '4점 이상(Positive)' 중에서 가장 최근 것을 가져가야 함.
# 나머지(4점 이상의 과거 데이터 + 3점 이하의 모든 데이터)는 Train으로 보냄.

def smart_split(g):
    # 1. 기본적으로 모두 'train'으로 설정
    g['split'] = 'train'

    # 2. 4점 이상인 데이터(Positive)만 추출
    pos_mask = g['rating'] >= 4.0
    pos_indices = g[pos_mask].index

    # Positive 데이터가 최소 3개 이상이어야 Train/Val/Test로 나눌 수 있음
    # (k-core=10을 해도 4점 미만이 많으면 Positive는 3개 미만일 수 있음)
    if len(pos_indices) >= 3:
        # 가장 최근 Positive -> Test
        g.loc[pos_indices[-1], 'split'] = 'test'
        # 그 전 Positive -> Val
        g.loc[pos_indices[-2], 'split'] = 'val'
        # 나머지는 그대로 'train' 유지

    return g

print("\n데이터 분할 중 (Positive 위주 Split)...")
df = df.groupby("userId", group_keys=False).apply(smart_split)

# =========================================================
# 4. ID 재매핑 (Unified ID Mapping)
# =========================================================
# K-core 이후 살아남은 유저/아이템에 대해서만 ID를 새로 매핑 (0부터 시작하도록)
unique_users = df['userId'].unique()
unique_items = df['movieId'].unique()

user2idx = {u: i for i, u in enumerate(unique_users)}
item2idx = {i: j for j, i in enumerate(unique_items)}

df['user'] = df['userId'].map(user2idx)
df['item'] = df['movieId'].map(item2idx)

print(f"ID 매핑 완료. (User: {len(user2idx)}, Item: {len(item2idx)})")

# =========================================================
# 5. 데이터 저장 (rating 컬럼 포함!)
# =========================================================
columns_to_save = ['user', 'item', 'rating']

train_df = df[df['split']=='train'][columns_to_save]
val_df   = df[df['split']=='val'][columns_to_save]
test_df  = df[df['split']=='test'][columns_to_save]

# 경로가 존재하지 않으면 생성 로직이 맨 위에 있으나, 파일명 변경 시 주의
train_df.to_csv(f"{save_path}/train.csv", index=False)
val_df.to_csv(f"{save_path}/val.csv", index=False)
test_df.to_csv(f"{save_path}/test.csv", index=False)

# 매핑 정보 저장
with open(f"{save_path}/user2idx.pkl", 'wb') as f: pickle.dump(user2idx, f)
with open(f"{save_path}/item2idx.pkl", 'wb') as f: pickle.dump(item2idx, f)

print("\n=== 저장 완료 ===")
print(f"저장 경로: {save_path}")
print(f"Train 데이터 수: {len(train_df)} (Positive + Negative 포함)")
print(f"Val 데이터 수: {len(val_df)} (Positive Only)")
print(f"Test 데이터 수: {len(test_df)} (Positive Only)")

데이터 로드 중...
원본 데이터 인터랙션 수: 99795
원본 유저 수: 671, 원본 아이템 수: 9012

=== 10-Core Filtering 시작 ===
필터링 진행 중... 남은 데이터: 81809
필터링 진행 중... 남은 데이터: 81800
필터링 진행 중... 남은 데이터: 81800

=== 10-Core Filtering 완료 ===
최종 인터랙션 수: 81800
최종 유저 수: 670, 최종 아이템 수: 2241
평점 분포:
rating
0.5      741
1.0     2304
1.5     1192
2.0     5532
2.5     3299
3.0    16294
3.5     8288
4.0    24198
4.5     6523
5.0    13429
Name: count, dtype: int64

데이터 분할 중 (Positive 위주 Split)...


/tmp/ipython-input-882659908.py:93: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("userId", group_keys=False).apply(smart_split)


ID 매핑 완료. (User: 670, Item: 2241)

=== 저장 완료 ===
저장 경로: /content/drive/MyDrive/unstructured/k10_filtered
Train 데이터 수: 80472 (Positive + Negative 포함)
Val 데이터 수: 664 (Positive Only)
Test 데이터 수: 664 (Positive Only)


In [11]:
import pandas as pd

# =========================================================
# 0. 데이터 준비 (앞선 코드 실행 후라고 가정)
# =========================================================
# df: 방금 전 k=10 필터링을 마친 데이터프레임
# 원본 데이터는 다시 로드해서 비교합니다.
print("--- 원본 데이터 다시 로드 중 ---")
original_path = f"{base_path}/rating_small_data_final_with_k_core_filtering.csv" # 기존 경로 사용
original_df = pd.read_csv(original_path)

# =========================================================
# 1. 기초 통계량 계산 함수
# =========================================================
def calculate_stats(data, name="Data"):
    n_users = data['userId'].nunique()
    n_items = data['movieId'].nunique()
    n_interactions = len(data)

    # 희소성 (Sparsity) 계산
    # 공식: 1 - (상호작용 수 / (유저 수 * 아이템 수))
    matrix_size = n_users * n_items
    if matrix_size == 0: sparsity = 1.0 # 0으로 나누기 방지
    else: sparsity = 1 - (n_interactions / matrix_size)

    # 밀도 (Density) = 1 - 희소성
    density = (n_interactions / matrix_size) * 100 if matrix_size != 0 else 0

    return {
        "name": name,
        "n_users": n_users,
        "n_items": n_items,
        "n_interactions": n_interactions,
        "sparsity": sparsity,
        "density": density
    }

# =========================================================
# 2. 통계 계산 및 비교
# =========================================================
org_stats = calculate_stats(original_df, "원본 데이터")
new_stats = calculate_stats(df, "필터링 데이터 (k=10)")

print("\n" + "="*50)
print(f"📊 데이터 변화 분석 보고서 (k={k_core})")
print("="*50)

# 1) 수량 비교
print(f"\n1. 데이터 수량 변화")
print(f"   - 유저 수 : {org_stats['n_users']:,}명 ➡️ {new_stats['n_users']:,}명 (생존율: {new_stats['n_users']/org_stats['n_users']*100:.2f}%)")
print(f"   - 아이템 수: {org_stats['n_items']:,}개 ➡️ {new_stats['n_items']:,}개 (생존율: {new_stats['n_items']/org_stats['n_items']*100:.2f}%)")
print(f"   - 상호작용 : {org_stats['n_interactions']:,}건 ➡️ {new_stats['n_interactions']:,}건 (생존율: {new_stats['n_interactions']/org_stats['n_interactions']*100:.2f}%)")

# 2) 희소성 비교
print(f"\n2. 매트릭스 밀도 변화 (학습 용이성)")
print(f"   - 원본 희소성(Sparsity): {org_stats['sparsity']*100:.4f}% (비어있는 공간의 비율)")
print(f"   - 최종 희소성(Sparsity): {new_stats['sparsity']*100:.4f}%")
print(f"   - 최종 밀도(Density)   : {new_stats['density']:.4f}%")

print("\n" + "-"*50)

# 3) 해석 및 조언
print("💡 해석 가이드:")
if new_stats['sparsity'] < 0.99:
    print("   >> 밀도가 매우 높습니다(희소성 < 99%). 모델이 학습하기 아주 좋은 환경입니다.")
    print("      다만, 너무 쉬운 데이터만 남았을 수 있으니 Test 성능이 과하게 높게 나올 수 있습니다.")
elif new_stats['sparsity'] > 0.999:
    print("   >> 여전히 매우 희소합니다(희소성 > 99.9%). 모델 학습이 조금 어려울 수 있습니다.")
else:
    print("   >> 적절한 수준의 희소성(99.0% ~ 99.9%)입니다. 일반적인 연구 환경과 비슷합니다.")

if new_stats['n_interactions'] / org_stats['n_interactions'] < 0.5:
    print(f"\n⚠️ 주의: 데이터의 50% 이상({100 - new_stats['n_interactions']/org_stats['n_interactions']*100:.1f}%)이 필터링되었습니다.")
    print("   데이터 손실이 큽니다. 만약 결과가 좋지 않다면 k값을 5 정도로 낮추는 것을 고려하세요.")
else:
    print(f"\n✅ 양호: 충분한 데이터({new_stats['n_interactions']/org_stats['n_interactions']*100:.1f}%)가 유지되었습니다.")
print("="*50)

--- 원본 데이터 다시 로드 중 ---

📊 데이터 변화 분석 보고서 (k=10)

1. 데이터 수량 변화
   - 유저 수 : 605명 ➡️ 670명 (생존율: 110.74%)
   - 아이템 수: 1,234개 ➡️ 2,241개 (생존율: 181.60%)
   - 상호작용 : 38,405건 ➡️ 81,800건 (생존율: 212.99%)

2. 매트릭스 밀도 변화 (학습 용이성)
   - 원본 희소성(Sparsity): 94.8558% (비어있는 공간의 비율)
   - 최종 희소성(Sparsity): 94.5520%
   - 최종 밀도(Density)   : 5.4480%

--------------------------------------------------
💡 해석 가이드:
   >> 밀도가 매우 높습니다(희소성 < 99%). 모델이 학습하기 아주 좋은 환경입니다.
      다만, 너무 쉬운 데이터만 남았을 수 있으니 Test 성능이 과하게 높게 나올 수 있습니다.

✅ 양호: 충분한 데이터(213.0%)가 유지되었습니다.


In [12]:
import pandas as pd
import numpy as np
import os
import pickle

# =========================================================
# 1. 환경 설정 및 데이터 로드
# =========================================================
base_path = "/content/drive/MyDrive/unstructured"

# [입력 경로 확인 필요] Raw 데이터 혹은 필터링 전 데이터를 지정해주세요.
input_rating_path = f"{base_path}/rating_small_data_final.csv"
save_path = f"{base_path}/k5_filtered" # 경로 구분 위해 폴더명 살짝 변경 추천

if not os.path.exists(save_path):
    os.makedirs(save_path)

print("데이터 로드 중...")
df = pd.read_csv(input_rating_path)

print(f"원본 데이터 인터랙션 수: {len(df)}")
print(f"원본 유저 수: {df['userId'].nunique()}, 원본 아이템 수: {df['movieId'].nunique()}")

# =========================================================
# [NEW] 1.5. K-core Filtering (k=10)
# =========================================================
# 유저와 아이템 모두 최소 10개 이상의 인터랙션이 있는 데이터만 남김
# 한쪽을 지우면 다른 쪽 count가 변하므로, 변화가 없을 때까지 반복 수행해야 함

k_core = 5
print(f"\n=== {k_core}-Core Filtering 시작 ===")

while True:
    start_len = len(df)

    # 1. 유저 필터링 (count < k 제거)
    user_counts = df['userId'].value_counts()
    valid_users = user_counts[user_counts >= k_core].index
    df = df[df['userId'].isin(valid_users)]

    # 2. 아이템 필터링 (count < k 제거)
    item_counts = df['movieId'].value_counts()
    valid_items = item_counts[item_counts >= k_core].index
    df = df[df['movieId'].isin(valid_items)]

    end_len = len(df)

    print(f"필터링 진행 중... 남은 데이터: {end_len}")

    # 더 이상 제거된 데이터가 없으면 반복 종료
    if start_len == end_len:
        break

print(f"\n=== {k_core}-Core Filtering 완료 ===")
print(f"최종 인터랙션 수: {len(df)}")
print(f"최종 유저 수: {df['userId'].nunique()}, 최종 아이템 수: {df['movieId'].nunique()}")
print(f"평점 분포:\n{df['rating'].value_counts().sort_index()}")

# =========================================================
# 2. 정렬 (Sorting)
# =========================================================
if 'timestamp' in df.columns:
    df = df.sort_values(['userId', 'timestamp'])
else:
    df = df.sort_values(['userId'])

# =========================================================
# 3. 데이터 분할 (Smart Leave-one-out)
# =========================================================
# 전략: Val/Test는 무조건 '4점 이상(Positive)' 중에서 가장 최근 것을 가져가야 함.
# 나머지(4점 이상의 과거 데이터 + 3점 이하의 모든 데이터)는 Train으로 보냄.

def smart_split(g):
    # 1. 기본적으로 모두 'train'으로 설정
    g['split'] = 'train'

    # 2. 4점 이상인 데이터(Positive)만 추출
    pos_mask = g['rating'] >= 4.0
    pos_indices = g[pos_mask].index

    # Positive 데이터가 최소 3개 이상이어야 Train/Val/Test로 나눌 수 있음
    # (k-core=10을 해도 4점 미만이 많으면 Positive는 3개 미만일 수 있음)
    if len(pos_indices) >= 3:
        # 가장 최근 Positive -> Test
        g.loc[pos_indices[-1], 'split'] = 'test'
        # 그 전 Positive -> Val
        g.loc[pos_indices[-2], 'split'] = 'val'
        # 나머지는 그대로 'train' 유지

    return g

print("\n데이터 분할 중 (Positive 위주 Split)...")
df = df.groupby("userId", group_keys=False).apply(smart_split)

# =========================================================
# 4. ID 재매핑 (Unified ID Mapping)
# =========================================================
# K-core 이후 살아남은 유저/아이템에 대해서만 ID를 새로 매핑 (0부터 시작하도록)
unique_users = df['userId'].unique()
unique_items = df['movieId'].unique()

user2idx = {u: i for i, u in enumerate(unique_users)}
item2idx = {i: j for j, i in enumerate(unique_items)}

df['user'] = df['userId'].map(user2idx)
df['item'] = df['movieId'].map(item2idx)

print(f"ID 매핑 완료. (User: {len(user2idx)}, Item: {len(item2idx)})")

# =========================================================
# 5. 데이터 저장 (rating 컬럼 포함!)
# =========================================================
columns_to_save = ['user', 'item', 'rating']

train_df = df[df['split']=='train'][columns_to_save]
val_df   = df[df['split']=='val'][columns_to_save]
test_df  = df[df['split']=='test'][columns_to_save]

# 경로가 존재하지 않으면 생성 로직이 맨 위에 있으나, 파일명 변경 시 주의
train_df.to_csv(f"{save_path}/train.csv", index=False)
val_df.to_csv(f"{save_path}/val.csv", index=False)
test_df.to_csv(f"{save_path}/test.csv", index=False)

# 매핑 정보 저장
with open(f"{save_path}/user2idx.pkl", 'wb') as f: pickle.dump(user2idx, f)
with open(f"{save_path}/item2idx.pkl", 'wb') as f: pickle.dump(item2idx, f)

print("\n=== 저장 완료 ===")
print(f"저장 경로: {save_path}")
print(f"Train 데이터 수: {len(train_df)} (Positive + Negative 포함)")
print(f"Val 데이터 수: {len(val_df)} (Positive Only)")
print(f"Test 데이터 수: {len(test_df)} (Positive Only)")

데이터 로드 중...
원본 데이터 인터랙션 수: 99795
원본 유저 수: 671, 원본 아이템 수: 9012

=== 5-Core Filtering 시작 ===
필터링 진행 중... 남은 데이터: 89927
필터링 진행 중... 남은 데이터: 89927

=== 5-Core Filtering 완료 ===
최종 인터랙션 수: 89927
최종 유저 수: 671, 최종 아이템 수: 3485
평점 분포:
rating
0.5      848
1.0     2752
1.5     1385
2.0     6342
2.5     3727
3.0    18099
3.5     9199
4.0    26336
4.5     7018
5.0    14221
Name: count, dtype: int64

데이터 분할 중 (Positive 위주 Split)...


/tmp/ipython-input-3602094512.py:93: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("userId", group_keys=False).apply(smart_split)


ID 매핑 완료. (User: 671, Item: 3485)

=== 저장 완료 ===
저장 경로: /content/drive/MyDrive/unstructured/k5_filtered
Train 데이터 수: 88595 (Positive + Negative 포함)
Val 데이터 수: 666 (Positive Only)
Test 데이터 수: 666 (Positive Only)


In [13]:
import pandas as pd

# =========================================================
# 0. 데이터 준비 (앞선 코드 실행 후라고 가정)
# =========================================================
# df: 방금 전 k=10 필터링을 마친 데이터프레임
# 원본 데이터는 다시 로드해서 비교합니다.
print("--- 원본 데이터 다시 로드 중 ---")
original_path = f"{base_path}/rating_small_data_final_with_k_core_filtering.csv" # 기존 경로 사용
original_df = pd.read_csv(original_path)

# =========================================================
# 1. 기초 통계량 계산 함수
# =========================================================
def calculate_stats(data, name="Data"):
    n_users = data['userId'].nunique()
    n_items = data['movieId'].nunique()
    n_interactions = len(data)

    # 희소성 (Sparsity) 계산
    # 공식: 1 - (상호작용 수 / (유저 수 * 아이템 수))
    matrix_size = n_users * n_items
    if matrix_size == 0: sparsity = 1.0 # 0으로 나누기 방지
    else: sparsity = 1 - (n_interactions / matrix_size)

    # 밀도 (Density) = 1 - 희소성
    density = (n_interactions / matrix_size) * 100 if matrix_size != 0 else 0

    return {
        "name": name,
        "n_users": n_users,
        "n_items": n_items,
        "n_interactions": n_interactions,
        "sparsity": sparsity,
        "density": density
    }

# =========================================================
# 2. 통계 계산 및 비교
# =========================================================
org_stats = calculate_stats(original_df, "원본 데이터")
new_stats = calculate_stats(df, "필터링 데이터 (k=10)")

print("\n" + "="*50)
print(f"📊 데이터 변화 분석 보고서 (k={k_core})")
print("="*50)

# 1) 수량 비교
print(f"\n1. 데이터 수량 변화")
print(f"   - 유저 수 : {org_stats['n_users']:,}명 ➡️ {new_stats['n_users']:,}명 (생존율: {new_stats['n_users']/org_stats['n_users']*100:.2f}%)")
print(f"   - 아이템 수: {org_stats['n_items']:,}개 ➡️ {new_stats['n_items']:,}개 (생존율: {new_stats['n_items']/org_stats['n_items']*100:.2f}%)")
print(f"   - 상호작용 : {org_stats['n_interactions']:,}건 ➡️ {new_stats['n_interactions']:,}건 (생존율: {new_stats['n_interactions']/org_stats['n_interactions']*100:.2f}%)")

# 2) 희소성 비교
print(f"\n2. 매트릭스 밀도 변화 (학습 용이성)")
print(f"   - 원본 희소성(Sparsity): {org_stats['sparsity']*100:.4f}% (비어있는 공간의 비율)")
print(f"   - 최종 희소성(Sparsity): {new_stats['sparsity']*100:.4f}%")
print(f"   - 최종 밀도(Density)   : {new_stats['density']:.4f}%")

print("\n" + "-"*50)

# 3) 해석 및 조언
print("💡 해석 가이드:")
if new_stats['sparsity'] < 0.99:
    print("   >> 밀도가 매우 높습니다(희소성 < 99%). 모델이 학습하기 아주 좋은 환경입니다.")
    print("      다만, 너무 쉬운 데이터만 남았을 수 있으니 Test 성능이 과하게 높게 나올 수 있습니다.")
elif new_stats['sparsity'] > 0.999:
    print("   >> 여전히 매우 희소합니다(희소성 > 99.9%). 모델 학습이 조금 어려울 수 있습니다.")
else:
    print("   >> 적절한 수준의 희소성(99.0% ~ 99.9%)입니다. 일반적인 연구 환경과 비슷합니다.")

if new_stats['n_interactions'] / org_stats['n_interactions'] < 0.5:
    print(f"\n⚠️ 주의: 데이터의 50% 이상({100 - new_stats['n_interactions']/org_stats['n_interactions']*100:.1f}%)이 필터링되었습니다.")
    print("   데이터 손실이 큽니다. 만약 결과가 좋지 않다면 k값을 5 정도로 낮추는 것을 고려하세요.")
else:
    print(f"\n✅ 양호: 충분한 데이터({new_stats['n_interactions']/org_stats['n_interactions']*100:.1f}%)가 유지되었습니다.")
print("="*50)

--- 원본 데이터 다시 로드 중 ---

📊 데이터 변화 분석 보고서 (k=5)

1. 데이터 수량 변화
   - 유저 수 : 605명 ➡️ 671명 (생존율: 110.91%)
   - 아이템 수: 1,234개 ➡️ 3,485개 (생존율: 282.41%)
   - 상호작용 : 38,405건 ➡️ 89,927건 (생존율: 234.15%)

2. 매트릭스 밀도 변화 (학습 용이성)
   - 원본 희소성(Sparsity): 94.8558% (비어있는 공간의 비율)
   - 최종 희소성(Sparsity): 96.1544%
   - 최종 밀도(Density)   : 3.8456%

--------------------------------------------------
💡 해석 가이드:
   >> 밀도가 매우 높습니다(희소성 < 99%). 모델이 학습하기 아주 좋은 환경입니다.
      다만, 너무 쉬운 데이터만 남았을 수 있으니 Test 성능이 과하게 높게 나올 수 있습니다.

✅ 양호: 충분한 데이터(234.2%)가 유지되었습니다.
